In [1]:
import sys
sys.path.append('C:\\Users\\edpou\\Desktop\\PyTreeNet-main')
import pytreenet as ptn
import numpy as np


# Operators 
(operator , node_identifiers)
- op.to_matrix
- op.to_tensor

In [3]:
op = ptn.NumericOperator( ptn.crandn((3,3,3,3)), ["apple", "pear"] )
op.operator.shape , op.node_identifiers

((3, 3, 3, 3), ['apple', 'pear'])

In [4]:
op = op.to_matrix()
op.operator.shape

(9, 9)

In [5]:
op = op.to_tensor(3) # when all nodes have the same open dimension
op.operator.shape 

(3, 3, 3, 3)

In [9]:
"""
                |2
                |
                r
               / \\
         3|  5/  6\\   |4
          |  /     \\  |
           c1        c2
"""

'\n                |2\n                |\n                r\n               / \\\n         3|  5/  6\\   |4\n          |  /     \\  |\n           c1        c2\n'

In [6]:
ttn = ptn.random_small_ttns()
op = ptn.NumericOperator( ptn.crandn((24,24)), ["root", "c1", "c2"])
op = op.to_tensor(ttn = ttn)
print(op.operator.shape)

# check unitarity 
op.is_unitary()

(2, 3, 4, 2, 3, 4)


False

# Common Operators

In [7]:
X , Y , Z = ptn.pauli_matrices()
creation_op, annihilation_op, number_op = ptn.bosonic_operators( dimension = 2 )
swap = ptn.swap_gate( dimension = 2 )
H = ptn.random_hermitian_matrix(size=2)

# Tensorproduct
tp = dict { "node_identifiers" : numpy.ndarray }
- tp.into_operator
- tp.pad_with_identities(ttn)

In [15]:
"""
                |2
                |
                r
               / \\
         3|  5/  6\\   |4
          |  /     \\  |
           c1        c2
"""
ttn = ptn.random_small_ttns()

In [17]:
array_dict = {"root": "a^dagger",
              "c1": "a"}
tp = ptn.TensorProduct(array_dict)

array_dict = {"root": ptn.crandn((2,2)),
              "c1": ptn.crandn((3,3))}
tp = ptn.TensorProduct(array_dict)

In [23]:
tp = tp.pad_with_identities(ttn)

In [24]:
operators = tp.into_operator() 
operators.operator.shape , operators.node_identifiers

((24, 24), ['root', 'c1', 'c2'])

In [14]:
# List of single site operators
op1 = ptn.NumericOperator(ptn.crandn((2,2)), "root")
op2 = ptn.NumericOperator(ptn.crandn((3,3)), "c1")

tp = ptn.TensorProduct.from_operators([op1, op2])


In [15]:
tps = tp.pad_with_identities(ttn,symbolic=True)
print(tps["c2"])

tp = tp.pad_with_identities(ttn)
print(tp["c2"])

I4
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


In [16]:
op = tp.into_operator( order=['root', 'c1', 'c2'] ) 
op = op.to_tensor(ttn=ttn)
print(op.operator.shape)
print(op.node_identifiers)

# order should be specified, since the
# tensor product is not commutative.
print("defailt order :" , list(tp.keys()) )

# turn tp into exp(factor*operator) with defaulft order
op = tp.exp( factor=2 )

(2, 3, 4, 2, 3, 4)
['root', 'c1', 'c2']
defailt order : ['root', 'c1', 'c2']


In [17]:
ttn = ptn.random_small_ttns()
tp = ptn.random_tensor_product(ttn, num_factors= 1)
# 1 =< num_factors =< len(ttn.nodes)
tp

TypeError: random_tensor_product() got an unexpected keyword argument 'num_factors'

# Hamiltonian
- H = List [ tp , tp , .... ]
- H.to_matrix = operator
- H.to_tensor = operator

In [18]:
X,Y,Z = ptn.pauli_matrices()

dict1 = {"site1" : X , "site2" : Z@Y }
dict2 = {"site3" : X@Y , "site4" : Z }

tp1 = ptn.TensorProduct(dict1)
tp2 = ptn.TensorProduct(dict2)
H = ptn.Hamiltonian([tp1,tp2])
# or
H = ptn.Hamiltonian([dict1,dict2])
#######
dict3 = {"site5" : Y }
tp3 = ptn.TensorProduct(dict3)

H.__add__(tp3)
######
tp4 = ptn.TensorProduct({"site6" : Z@X })
tp5 = ptn.TensorProduct({"site7" : Z@Y })
# H.add_term(tp4)
H.add_multiple_terms([tp4,tp5])

#######
tp6 = ptn.TensorProduct({"site7" : Z@X })
H2 = ptn.Hamiltonian([tp6])

H.__add__(H2)
#######
# check if all node ids in H are in ttn
H.is_compatible_with(ttn)
H.perform_compatibility_checks(ptn.PadMode.risky , ttn ) # does not check compatibility

H.terms[0] == tp1 , H.terms[2] == tp3

(True, True)

In [ ]:
ttn = ptn.random_small_ttns()
tp1 = ptn.TensorProduct({"root": "A", "c2": "C"})
H = ptn.Hamiltonian( tp1 , conversion_dictionary = {"A": ptn.crandn((2,2)), "B": ptn.crandn((3,3))})
H = H.pad_with_identities(ttn , symbolic = False)
# len(H.terms) == 1

In [ ]:
ttn = ptn.random_small_ttns()
tp1 = ptn.random_tensor_product(ttn, num_factors= len(ttn))
tp2 = ptn.random_tensor_product(ttn, num_factors= 2)
H = ptn.Hamiltonian([tp1,tp2])
H = H.pad_with_identities(ttn , symbolic = False)

H_op = H.to_matrix(ttn) # order = list(ttn.nodes.keys())
print(H_op.operator.shape , H_op.node_identifiers)
H_tn = H.to_tensor(ttn)
print(H_tn.operator.shape , H_tn.node_identifiers)

(24, 24) ['root', 'c1', 'c2']
(2, 3, 4, 2, 3, 4) ['root', 'c1', 'c2']


In [ ]:
tp = ptn.TensorProduct( {"root" : "A" , "c1" : "B" , "c2" : "C" } )
H = ptn.Hamiltonian( tp , conversion_dictionary = {"A": ptn.crandn((2,2)), "B": ptn.crandn((3,3)), "C": ptn.crandn((4,4))})
ttn = ptn.random_small_ttns()
H_op  = H.to_matrix(ttn)
print ( H_op.operator.shape , H_op.node_identifiers )
H_op = H.to_tensor(ttn)
print ( H_op.operator.shape , H_op.node_identifiers )

(24, 24) ['root', 'c1', 'c2']
(2, 3, 4, 2, 3, 4) ['root', 'c1', 'c2']


In [ ]:
possible_operators = [X, X@Y, Z@X, Z, Y, X@Y , Z@Z, Y@X, Z@Y ] 
sites = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J"]
List = ptn.random_symbolic_terms(
                                 num_of_terms=4, 
                                 possible_operators=possible_operators, 
                                 sites=sites, 
                                 min_num_sites=2, 
                                 max_num_sites=4, 
                                 seed=None)

# List = [tp1, tp2, tp3, tp4]
# 2 <= len(tp1) <= 4
op1 = List[0].into_operator()
print(op1.operator.shape , op1.node_identifiers)

H = ptn.Hamiltonian(List)
H.terms[0] == List[0]

In [25]:
List = ptn.random_terms(num_of_terms = 4,
                        possible_operators = possible_operators,
                        sites = sites,
                        min_strength = 1,
                        max_strength = 4,
                        min_num_sites= 2,
                        max_num_sites= 4)

# List = [dict1, dict2, dict3, dict4]
# 2 <= len(dict) <= 4
# dict = {"site" : (random strength)*operator , ... }

In [28]:
tp = ptn.TensorProduct(List[0])

In [24]:
H = ptn.create_single_site_hamiltonian(ttn, 
                                       local_operator = "X", 
                                       conversion_dict = {"X" : ptn.pauli_matrices()[0]})
H.terms 


[{'root': 'X'}, {'c1': 'X'}, {'c2': 'X'}]

# ttns

In [ ]:
ttn = ptn.random_small_ttns()

In [ ]:
# <TTNS|TTNS>
print( ttn.scalar_product() )

# <TTNS| operator(nd.array) , node_id |TTNS>
# Assumes the node has only one open leg. 
print( ttn.single_site_operator_expectation_value("root", ptn.crandn((2,2))) )

# <TTNS|tp|TTNS>
tp = ptn.random_tensor_product(ttn, num_factors= 2)
print( ttn.operator_expectation_value(tp) )

(198.4911323158384+3.552713678800501e-15j)
(269.62646114273537-48.52127247401511j)
(187.56320099276218+63.65793184544422j)


In [ ]:
ham_ttno.completely_contract_tree(to_copy=True)[1]

(['site3', 'site2', 'site1', 'site0', 'site4', 'site5', 'site6'],
 dict_keys(['site3', 'site2', 'site1', 'site0', 'site4', 'site5', 'site6']))

In [ ]:
hamiltonian_matrix = ham_ttno.completely_contract_tree(to_copy=True)[0]
hamiltonian_matrix = np.transpose(hamiltonian_matrix,(1,3,5,7,9,11,13,0,2,4,6,8,10,12))
hamiltonian_matrix = np.reshape(hamiltonian_matrix,(2**num_sites,2**num_sites))

hamiltonian_matrix2 = hamiltonian.to_matrix(mps).operator

print(np.allclose(hamiltonian_matrix, hamiltonian_matrix2))

NameError: name 'ham_ttno' is not defined

In [ ]:
hamiltonian_tensor = ham_ttno.completely_contract_tree(to_copy=True)[0]
hamiltonian_tensor = np.transpose(hamiltonian_tensor,(1,3,5,7,9,11,13,0,2,4,6,8,10,12))

hamiltonian_tensor2 = hamiltonian.to_tensor(mps).operator
print(np.allclose(hamiltonian_tensor, hamiltonian_tensor2))

True


In [ ]:
mps.nodes.keys()

dict_keys(['site3', 'site2', 'site1', 'site0', 'site4', 'site5', 'site6'])

In [ ]:
hamiltonian_tensor = ham_ttno.completely_contract_tree(to_copy=True)[0]

hamiltonian_tensor2 = hamiltonian.to_tensor(mps).operator
hamiltonian_tensor2 = np.transpose(hamiltonian_tensor2,(0,7,1,8,2,9,3,10,4,11,5,12,6,13))

print(np.allclose(hamiltonian_tensor, hamiltonian_tensor2))

True
